# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Python\cassandra_music_db


In [6]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            print(line)
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


', 'PUT', 'NextSong', '1.54047E+12', '977', 'Feel It', '200', '1.54359E+12', '85']
['Sugababes', 'Logged In', 'Kinsley', 'F', '33', 'Young', '275.93098', 'paid', 'Red Bluff, CA', 'PUT', 'NextSong', '1.54047E+12', '977', 'Overload', '200', '1.54359E+12', '85']
['Gyptian', 'Logged In', 'Rylan', 'M', '13', 'George', '233.09016', 'paid', 'Birmingham-Hoover, AL', 'PUT', 'NextSong', '1.54102E+12', '1076', 'Hold You', '200', '1.54359E+12', '16']
['Zion / Akon', 'Logged In', 'Rylan', 'M', '14', 'George', '233.19465', 'paid', 'Birmingham-Hoover, AL', 'PUT', 'NextSong', '1.54102E+12', '1076', 'The Way She Moves', '200', '1.54359E+12', '16']
['Transvision Vamp', 'Logged In', 'Rylan', 'M', '15', 'George', '209.91955', 'paid', 'Birmingham-Hoover, AL', 'PUT', 'NextSong', '1.54102E+12', '1076', 'I Want Your Love', '200', '1.54359E+12', '16']
['Ry Cooder', 'Logged In', 'Chloe', 'F', '37', 'Cuevas', '201.56036', 'paid', 'San Francisco-Oakland-Hayward, CA', 'PUT', 'NextSong', '1.54094E+12', '1096', 'Cor

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [8]:
def insert_data(file, session, query, indicies, dtypes):
    '''
    Creating pipeline to insert data into a Apache Cassandra table from CSV file.
    args:
        file(str): CSV file name
        session(object): Apache Cassandra session
        query(str): INSERT query statement 
        indicies(list): list of indicies to pass to INSERT statement
        dtypes(list): list of data types to convert indices values
    
    return: 
        None      
    '''
    #check both are of same size else stop executing the rest and throw an error
    assert(len(indicies) == len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple(dtype(line[x]) for x, dtype in zip(indicies, dtypes)))

## Create queries to ask the following three questions of the data

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [9]:
# Create Table for Query 1
query = "CREATE TABLE IF NOT EXISTS song_playlist_item_session"
query = query + '''(session_id int, 
                    item_in_session int, 
                    artist text, 
                    length float, 
                    song text, 
                    PRIMARY KEY(session_id, item_in_session))'''

try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Insert data into table song_playlist_item_sessioin

In [10]:
file = 'event_datafile_new.csv'
# Indices and dtypes for Query 1
indicies = [8, 3, 0, 5, 9]
dtypes = [int, int, str, float, str]

# INSERT query statement
query = "INSERT INTO song_playlist_item_session(session_id, item_in_session, artist, length, song)"
query = query + "VALUES(%s, %s, %s, %s,%s)"

# INSERT into the table
insert_data(file, session, query, indicies, dtypes)

#### Verify Query 1 - SELECT statement

In [11]:
query = """SELECT artist, length, song
           FROM song_playlist_item_session
           WHERE session_id = 338 AND item_in_session = 4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# collecting all rows in list as a tuple
allRows = [] 
for row in rows:
    allRows.append((row.artist, row.song,row.length))

In [12]:
# Create dataFrame for the result query
df = pd.DataFrame(allRows,columns=['artist','song','length'])
# Print top 5 rows in the dataFrame
df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [13]:
# Create Table for Query 2
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + '''(user_id int,
              session_id int, 
              item_in_session int,
              artist text,
              first_name text,
              last_name text,
              song text,
            PRIMARY KEY((user_id, session_id), item_in_session))'''

try:
    session.execute(query)
except Exception as e:
    print(e) 

#### Insert data into table song_playlist_session

In [14]:
# Indices and dtypes for Query 2
indicies = [10, 8, 3, 0, 1, 4, 9]
dtypes = [int, int, int, str, str,str, str]

# INSERT query statement
query = "INSERT INTO song_playlist_session(user_id, session_id, item_in_session, artist, first_name,last_name, song)"
query = query + "VALUES(%s, %s, %s, %s,%s, %s,%s)"

# INSERT into the table
insert_data(file, session, query, indicies, dtypes)

#### Verify Query 2 - SELECT statement

In [15]:
query = '''SELECT artist, song, first_name, last_name
           FROM song_playlist_session
           WHERE user_id = 10 AND session_id = 182
           order by item_in_session DESC'''

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# collecting all rows in list as a tuple
allRows = [] 
for row in rows:
    allRows.append((row.artist, row.song,row.first_name, row.last_name))

In [16]:
# Create dataFrame for the query result.
df1 = pd.DataFrame(allRows,columns=['artist','song','first_name','last_name'])
# print top 5 rows in the dataFrame
df1.head()

,artist,song,first_name,last_name
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz
1,Sebastien Tellier,Kilometer,Sylvie,Cruz
2,Three Drives,Greece 2000,Sylvie,Cruz
3,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
# Create Table for Query 3
query = "CREATE TABLE IF NOT EXISTS song_playlist_library"
query = query + '''(song text,
                    user_id int,
                    first_name text,
                    last_name text,
                    PRIMARY KEY(song, user_id))'''

try:
    session.execute(query)
except Exception as e:
    print(e) 

#### Insert data into table song_playlist_library

In [18]:
# Indices and dtypes for Query 3
indicies = [9, 10, 1, 4]
dtypes = [str, int, str, str]

# INSERT query statement
query = "INSERT INTO song_playlist_library(song, user_id, first_name, last_name)"
query = query + "VALUES(%s,%s, %s, %s)"

# INSERT into the table
insert_data(file, session, query, indicies, dtypes)

#### Verify Query 3 - SELECT statement

In [ ]:
query = '''SELECT first_name, last_name
           FROM song_playlist_library
           WHERE song = 'All Hands Against His Own' '''

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# collecting all rows in list as a tuple
allRows = []    
for row in rows:
    allRows.append((row.first_name, row.last_name))

In [ ]:
# Create dataFrame for the result query
df2 = pd.DataFrame(allRows,columns=['first_name','last_name'])
# print top 5 rows in the dataFrame
df2.head()

### Drop the tables before closing out the sessions

In [ ]:
def drop_table(table_names):
    
    '''
    This function helps in drop the tables in the KEYSAPCE.
    arg:
        table_names(list): list of table names to drop in the KEYSPACE
    return:
        None
    '''
    for table in table_names:
        query = "DROP TABLE "+table
        try:
            session.execute(query)
        except Exception as e:
            print(e)

In [ ]:
# List of table names to drop
table_names = ['song_playlist_item_session', 'song_playlist_session', 'song_playlist_library']

# Drop tables
drop_table(table_names)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()